1. взять любой набор данных для бинарной классификации (можно скачать один с https://archive.ics.uci.edu/ml/datasets.php)
2. сделать feature engineering
3. обучить любой классификатор (какой вам нравится)
4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
5. применить random negative sampling для построения классификатора в новых условиях
6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
7. поэкспериментировать с долей P на шаге 6 (как будет меняться качество модели при уменьшении/увеличении размера P)

In [1]:
import numpy as np
import pandas as pd

In [44]:
df = pd.read_csv('hepatitis.data', sep=',')
df

,2,30,2.1,1,2.2,2.3,2.4,2.5,1.1,2.6,2.7,2.8,2.9,2.10,1.00,85,18,4.0,?,1.2
0,2,50,1,1,2,1,2,2,1,2,2,2,2,2,0.90,135,42,3.5,?,1
1,2,78,1,2,2,1,2,2,2,2,2,2,2,2,0.70,96,32,4.0,?,1
2,2,31,1,?,1,2,2,2,2,2,2,2,2,2,0.70,46,52,4.0,80,1
3,2,34,1,2,2,2,2,2,2,2,2,2,2,2,1.00,?,200,4.0,?,1
4,2,34,1,2,2,2,2,2,2,2,2,2,2,2,0.90,95,28,4.0,75,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,1,46,1,2,2,1,1,1,2,2,2,1,1,1,7.60,?,242,3.3,50,2
150,2,44,1,2,2,1,2,2,2,1,2,2,2,2,0.90,126,142,4.3,?,2
151,2,61,1,1,2,1,1,2,1,1,2,1,2,2,0.80,75,20,4.1,?,2
152,2,53,2,1,2,1,2,2,2,2,1,1,2,1,1.50,81,19,4.1,48,2


In [45]:
df.rename(columns={'2':'target', '30':'age', '2.1':'sex', '1':'STEROID', '2.2':'ANTIVIRALS', '2.3':'FATIGUE',
                  '2.4':'MALAISE', '2.5':'ANOREXIA', '1.1':'LIVER BIG', '2.6':'LIVER FIRM', '2.7':'SPLEEN PALPABLE',
                  '2.8':'SPIDERS', '2.9':'ASCITES', '2.10':'VARICES', '1.00':'BILIRUBIN', '85':'ALK PHOSPHATE',
                  '18':'SGOT', '4.0':'ALBUMIN', '?':'PROTIME', '1.2':'HISTOLOGY'}, inplace=True)

In [46]:
df.columns

Index(['target', 'age', 'sex', 'STEROID', 'ANTIVIRALS', 'FATIGUE', 'MALAISE',
       'ANOREXIA', 'LIVER BIG', 'LIVER FIRM', 'SPLEEN PALPABLE', 'SPIDERS',
       'ASCITES', 'VARICES', 'BILIRUBIN', 'ALK PHOSPHATE', 'SGOT', 'ALBUMIN',
       'PROTIME', 'HISTOLOGY'],
      dtype='object')

In [47]:
df.describe()

,target,age,sex,ANTIVIRALS,HISTOLOGY
count,154.000000,154.000000,154.000000,154.000000,154.000000
mean,1.792208,41.272727,1.097403,1.844156,1.454545
std,0.407051,12.574106,0.297473,0.363891,0.499554
min,1.000000,7.000000,1.000000,1.000000,1.000000
25%,2.000000,32.000000,1.000000,2.000000,1.000000
50%,2.000000,39.000000,1.000000,2.000000,1.000000
75%,2.000000,50.000000,1.000000,2.000000,2.000000
max,2.000000,78.000000,2.000000,2.000000,2.000000


In [61]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   target           154 non-null    int64  
 1   age              154 non-null    int64  
 2   sex              154 non-null    int64  
 3   STEROID          154 non-null    int32  
 4   ANTIVIRALS       154 non-null    int64  
 5   FATIGUE          154 non-null    int32  
 6   MALAISE          154 non-null    int32  
 7   ANOREXIA         154 non-null    int32  
 8   LIVER BIG        154 non-null    int32  
 9   LIVER FIRM       154 non-null    int32  
 10  SPLEEN PALPABLE  154 non-null    int32  
 11  SPIDERS          154 non-null    int32  
 12  ASCITES          154 non-null    int32  
 13  VARICES          154 non-null    int32  
 14  BILIRUBIN        154 non-null    float64
 15  ALK PHOSPHATE    154 non-null    object 
 16  SGOT             154 non-null    object 
 17  ALBUMIN         

In [62]:
df

,target,age,sex,STEROID,ANTIVIRALS,FATIGUE,MALAISE,ANOREXIA,LIVER BIG,LIVER FIRM,SPLEEN PALPABLE,SPIDERS,ASCITES,VARICES,BILIRUBIN,ALK PHOSPHATE,SGOT,ALBUMIN,PROTIME,HISTOLOGY
0,2,50,1,1,2,1,2,2,1,2,2,2,2,2,0.9,135,42,3.5,?,1
1,2,78,1,2,2,1,2,2,2,2,2,2,2,2,0.7,96,32,4.0,?,1
2,2,31,1,2,1,2,2,2,2,2,2,2,2,2,0.7,46,52,4.0,80,1
3,2,34,1,2,2,2,2,2,2,2,2,2,2,2,1.0,?,200,4.0,?,1
4,2,34,1,2,2,2,2,2,2,2,2,2,2,2,0.9,95,28,4.0,75,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,1,46,1,2,2,1,1,1,2,2,2,1,1,1,7.6,?,242,3.3,50,2
150,2,44,1,2,2,1,2,2,2,1,2,2,2,2,0.9,126,142,4.3,?,2
151,2,61,1,1,2,1,1,2,1,1,2,1,2,2,0.8,75,20,4.1,?,2
152,2,53,2,1,2,1,2,2,2,2,1,1,2,1,1.5,81,19,4.1,48,2


In [58]:
for cat_colname in df.select_dtypes(include='object').columns:
    print(str(cat_colname) + '\n\n' + str(df[cat_colname].value_counts()) + '\n' + '*' * 100 + '\n')

STEROID

2    79
1    75
Name: STEROID, dtype: int64
****************************************************************************************************

FATIGUE

1    101
2     53
Name: FATIGUE, dtype: int64
****************************************************************************************************

MALAISE

2    93
1    61
Name: MALAISE, dtype: int64
****************************************************************************************************

ANOREXIA

2    122
1     32
Name: ANOREXIA, dtype: int64
****************************************************************************************************

LIVER BIG

2    130
1     24
Name: LIVER BIG, dtype: int64
****************************************************************************************************

LIVER FIRM

2    94
1    60
Name: LIVER FIRM, dtype: int64
****************************************************************************************************

SPLEEN PALPABLE

2    124
1     30
Name: SPLEEN PALP

In [65]:
for cat_colname in df.select_dtypes(include='object').columns:
    col_moda = df[cat_colname].mode()[0]
    df[cat_colname] = df[cat_colname].apply(lambda x: col_moda if (x=='?') else x)

In [67]:
for cat_colname in df.select_dtypes(include='int64').columns:
    col_moda = df[cat_colname].mode()[0]
    df[cat_colname] = df[cat_colname].apply(lambda x: col_moda if (x=='?') else x)

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   target           154 non-null    int64  
 1   age              154 non-null    int64  
 2   sex              154 non-null    int64  
 3   STEROID          154 non-null    int32  
 4   ANTIVIRALS       154 non-null    int64  
 5   FATIGUE          154 non-null    int32  
 6   MALAISE          154 non-null    int32  
 7   ANOREXIA         154 non-null    int32  
 8   LIVER BIG        154 non-null    int32  
 9   LIVER FIRM       154 non-null    int32  
 10  SPLEEN PALPABLE  154 non-null    int32  
 11  SPIDERS          154 non-null    int32  
 12  ASCITES          154 non-null    int32  
 13  VARICES          154 non-null    int32  
 14  BILIRUBIN        154 non-null    float64
 15  ALK PHOSPHATE    154 non-null    object 
 16  SGOT             154 non-null    object 
 17  ALBUMIN         

In [74]:
df.loc[df['ALK PHOSPHATE']=='? ', 'ALK PHOSPHATE'] = df['ALK PHOSPHATE'].mode()[0]
df.loc[df['SGOT']=='?', 'SGOT'] = df['SGOT'].mode()[0]
df.loc[df['ALBUMIN']=='?', 'ALBUMIN'] = df['ALBUMIN'].mode()[0]
df.loc[df['PROTIME']=='?', 'PROTIME'] = df['PROTIME'].mode()[0]
df.loc[df['HISTOLOGY']=='?', 'HISTOLOGY'] = df['HISTOLOGY'].mode()[0]

In [81]:
df = df.drop(columns=['ALK PHOSPHATE', 'PROTIME'], axis=0)

In [82]:
df

,target,age,sex,STEROID,ANTIVIRALS,FATIGUE,MALAISE,ANOREXIA,LIVER BIG,LIVER FIRM,SPLEEN PALPABLE,SPIDERS,ASCITES,VARICES,BILIRUBIN,SGOT,ALBUMIN,HISTOLOGY
0,2,50,1,1,2,1,2,2,1,2,2,2,2,2,0.9,42,3.5,1
1,2,78,1,2,2,1,2,2,2,2,2,2,2,2,0.7,32,4.0,1
2,2,31,1,2,1,2,2,2,2,2,2,2,2,2,0.7,52,4.0,1
3,2,34,1,2,2,2,2,2,2,2,2,2,2,2,1.0,200,4.0,1
4,2,34,1,2,2,2,2,2,2,2,2,2,2,2,0.9,28,4.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149,1,46,1,2,2,1,1,1,2,2,2,1,1,1,7.6,242,3.3,2
150,2,44,1,2,2,1,2,2,2,1,2,2,2,2,0.9,142,4.3,2
151,2,61,1,1,2,1,1,2,1,1,2,1,2,2,0.8,20,4.1,2
152,2,53,2,1,2,1,2,2,2,2,1,1,2,1,1.5,19,4.1,2


In [83]:
for cat_colname in df.select_dtypes(include='object').columns:
    df[cat_colname] = df[cat_colname].astype(float)

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 154 entries, 0 to 153
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   target           154 non-null    int64  
 1   age              154 non-null    int64  
 2   sex              154 non-null    int64  
 3   STEROID          154 non-null    int32  
 4   ANTIVIRALS       154 non-null    int64  
 5   FATIGUE          154 non-null    int32  
 6   MALAISE          154 non-null    int32  
 7   ANOREXIA         154 non-null    int32  
 8   LIVER BIG        154 non-null    int32  
 9   LIVER FIRM       154 non-null    int32  
 10  SPLEEN PALPABLE  154 non-null    int32  
 11  SPIDERS          154 non-null    int32  
 12  ASCITES          154 non-null    int32  
 13  VARICES          154 non-null    int32  
 14  BILIRUBIN        154 non-null    float64
 15  SGOT             154 non-null    float64
 16  ALBUMIN          154 non-null    float64
 17  HISTOLOGY       

In [87]:
df['target'] = df['target'].apply(lambda x: 1 if x==2 else 0)

In [89]:
df['target'].value_counts()

1    122
0     32
Name: target, dtype: int64

In [99]:
from sklearn.model_selection import train_test_split

x_data = df.drop(columns='target', axis=0)
y_data = df['target']

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [100]:
import xgboost as xgb

model = xgb.XGBClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

In [103]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score
f1 = f1_score(y_test, y_predict)
rec = recall_score(y_test, y_predict, average='binary')
prc = precision_score(y_test, y_predict, average='binary')
print(f'f1score {f1}\n recall {rec}\n precision {prc}')

f1score 0.9047619047619047
 recall 1.0
 precision 0.8260869565217391


In [107]:
mod_data = x_train.copy()
mod_data['label'] = y_train
mod_data = mod_data.reset_index(drop=True)

pos_ind = np.where(mod_data.iloc[:, -1].values == 1)[0]


np.random.shuffle(pos_ind)

perc = 0.25
pos_sample_len = int(np.ceil(perc * len(pos_ind)))

print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 26/103 as positives and unlabeling the rest


In [108]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample, 'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    97
 1    26
Name: class_test, dtype: int64


In [109]:
mod_data.head(10)

,age,sex,STEROID,ANTIVIRALS,FATIGUE,MALAISE,ANOREXIA,LIVER BIG,LIVER FIRM,SPLEEN PALPABLE,SPIDERS,ASCITES,VARICES,BILIRUBIN,SGOT,ALBUMIN,HISTOLOGY,label,class_test
0,28,2,2,2,1,1,2,2,1,2,2,2,2,1.8,420.0,3.3,1,1,-1
1,50,1,2,2,2,2,2,2,1,1,1,2,2,1.0,75.0,4.0,2,1,1
2,39,1,1,2,2,2,2,2,2,2,2,2,2,0.9,60.0,4.0,1,1,-1
3,42,1,2,2,2,2,2,2,2,2,2,2,2,0.9,63.0,4.7,1,1,-1
4,45,1,2,1,2,2,2,2,2,2,2,2,2,1.3,44.0,4.2,2,1,-1
5,51,1,1,2,1,1,1,2,1,1,1,2,1,4.6,269.0,3.9,2,1,-1
6,42,1,2,2,2,2,2,2,2,1,2,2,2,1.5,40.0,4.0,2,1,-1
7,20,2,1,2,1,1,1,1,1,1,1,2,2,2.3,68.0,3.9,1,1,-1
8,51,1,1,2,1,2,1,2,2,1,1,2,2,1.0,20.0,4.0,1,0,-1
9,38,1,2,2,2,2,2,2,2,2,2,2,2,0.7,42.0,4.1,2,1,1


In [110]:
mod_data = mod_data.sample(frac=1)


data_N = mod_data[mod_data['class_test'] == -1]
data_P = mod_data[mod_data['class_test'] == 1]

neg_sample = data_N[:data_P.shape[0]]
sample_test = data_N[data_P.shape[0]:]
pos_sample = data_P.copy()

print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(26, 19) (26, 19)


In [111]:
sample_train

,age,sex,STEROID,ANTIVIRALS,FATIGUE,MALAISE,ANOREXIA,LIVER BIG,LIVER FIRM,SPLEEN PALPABLE,SPIDERS,ASCITES,VARICES,BILIRUBIN,SGOT,ALBUMIN,HISTOLOGY,label,class_test
45,36,1,1,2,2,2,2,2,2,1,2,2,2,0.8,44.0,4.2,1,1,1
76,20,1,1,2,2,2,2,2,2,2,2,2,2,0.9,152.0,4.0,2,1,-1
50,38,1,1,1,2,2,2,1,1,2,2,2,2,0.7,28.0,4.2,1,1,-1
34,50,1,2,2,2,2,2,2,2,2,2,2,2,1.0,81.0,3.9,2,1,-1
37,66,1,2,2,1,2,2,2,2,2,2,2,2,1.2,53.0,4.3,1,1,1
29,34,1,1,2,1,2,2,1,1,2,1,2,2,1.0,46.0,4.4,1,1,1
120,61,1,1,2,1,1,2,1,1,2,1,2,2,0.8,20.0,4.1,2,1,1
22,52,1,1,1,2,2,2,2,2,2,2,2,2,0.7,55.0,4.0,1,1,1
98,37,1,2,2,1,2,2,2,2,2,1,2,2,0.6,28.0,4.2,1,0,-1
72,69,2,2,2,1,2,2,2,2,2,2,2,2,3.2,136.0,4.0,2,1,1


In [113]:
model = xgb.XGBClassifier()
sample_train.loc[sample_train['class_test'] == -1, 'class_test'] = 0

model.fit(sample_train.drop(columns=['class_test', 'label']), 
          sample_train['class_test'])

y_predict = model.predict(x_test)
f1 = f1_score(y_test, y_predict)
rec = recall_score(y_test, y_predict, average='binary')
prc = precision_score(y_test, y_predict, average='binary')
print(f'f1score {f1}\n recall {rec}\n precision {prc}')

f1score 0.36363636363636365
 recall 0.3157894736842105
 precision 0.42857142857142855


Попробуем увеличить процент до 50

In [114]:
mod_data = x_train.copy()
mod_data['label'] = y_train
mod_data = mod_data.reset_index(drop=True)

pos_ind = np.where(mod_data.iloc[:, -1].values == 1)[0]


np.random.shuffle(pos_ind)

perc = 0.5
pos_sample_len = int(np.ceil(perc * len(pos_ind)))

print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 52/103 as positives and unlabeling the rest


In [115]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample, 'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    71
 1    52
Name: class_test, dtype: int64


In [116]:
mod_data = mod_data.sample(frac=1)


data_N = mod_data[mod_data['class_test'] == -1]
data_P = mod_data[mod_data['class_test'] == 1]

neg_sample = data_N[:data_P.shape[0]]
sample_test = data_N[data_P.shape[0]:]
pos_sample = data_P.copy()

print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(52, 19) (52, 19)


In [117]:
model = xgb.XGBClassifier()
sample_train.loc[sample_train['class_test'] == -1, 'class_test'] = 0

model.fit(sample_train.drop(columns=['class_test', 'label']), 
          sample_train['class_test'])

y_predict = model.predict(x_test)
f1 = f1_score(y_test, y_predict)
rec = recall_score(y_test, y_predict, average='binary')
prc = precision_score(y_test, y_predict, average='binary')
print(f'f1score {f1}\n recall {rec}\n precision {prc}')

f1score 0.7058823529411765
 recall 0.631578947368421
 precision 0.8
